# Group 2 Phase 2 - Cats vs Dogs Detector (CaDoD) "FrankenNet" SECONDARY notebook

*Adapted from https://heartbeat.fritz.ai/basics-of-image-classification-with-pytorch-2f8973c51864*

### This notebook is a work in progress and will feed into our final submission! See Group2_Phase2.ipynb for earlier Phase 2 work.

## Team Members

* Ben Perkins
* Lauren Madar
* Mangesh Walimbe
* Samin Barghan

![](./other/team.png)

In [ ]:
# Import general modules
import os
import glob
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tarfile
from tqdm.notebook import tqdm
from PIL import Image
import warnings

# PyTorch Modules
import torch
import torch.nn
import torch.nn.functional as F
from torch.nn import ReLU
from torch import nn, optim
from torch.optim import Adam
from torch.autograd import Variable
import torch.utils
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import datasets, transforms, utils
from torchvision.io import read_image

In [ ]:
def extract_tar(file, path):
    """
    function to extract tar.gz files to specified location
    
    Args:
        file (str): path where the file is located
        path (str): path where you want to extract
    """
    with tarfile.open(file) as tar:
        files_extracted = 0
        for member in tqdm(tar.getmembers()):
            if os.path.isfile(path + member.name[1:]):
                continue
            else:
                tar.extract(member, path)
                files_extracted += 1
        tar.close()
        if files_extracted < 3:
            print('Files already exist')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/CatsNDogs/data/cadod/'

#extract_tar('/content/drive/MyDrive/Colab Notebooks/CatsNDogs/data/cadod.tar.gz', path)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CatsNDogs/cadod.csv')

In [ ]:
df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
0,0000b9fcba019d36,xclick,/m/0bt9lr,1,0.165000,0.903750,0.268333,0.998333,1,1,0,0,0,0.636250,0.903750,0.748750,0.165000,0.268333,0.506667,0.998333,0.661667
1,0000cb13febe0138,xclick,/m/0bt9lr,1,0.000000,0.651875,0.000000,0.999062,1,1,0,0,0,0.312500,0.000000,0.317500,0.651875,0.000000,0.410882,0.999062,0.999062
2,0005a9520eb22c19,xclick,/m/0bt9lr,1,0.094167,0.611667,0.055626,0.998736,1,1,0,0,0,0.487500,0.611667,0.243333,0.094167,0.055626,0.226296,0.998736,0.305942
3,0006303f02219b07,xclick,/m/0bt9lr,1,0.000000,0.999219,0.000000,0.998824,1,1,0,0,0,0.508594,0.999219,0.000000,0.478906,0.000000,0.375294,0.720000,0.998824
4,00064d23bf997652,xclick,/m/0bt9lr,1,0.240938,0.906183,0.000000,0.694286,0,0,0,0,0,0.678038,0.906183,0.240938,0.522388,0.000000,0.370000,0.424286,0.694286


In [ ]:
df.LabelName.unique()

array(['/m/0bt9lr', '/m/01yrx'], dtype=object)

# Create train_csv.csv

In [ ]:
def label_img(row):
    if row['LabelName'] == '/m/0bt9lr':
        return 1
    if row['LabelName'] == '/m/01yrx':
        return 0


In [ ]:
df2 = df
df2.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
0,0000b9fcba019d36,xclick,/m/0bt9lr,1,0.165000,0.903750,0.268333,0.998333,1,1,0,0,0,0.636250,0.903750,0.748750,0.165000,0.268333,0.506667,0.998333,0.661667
1,0000cb13febe0138,xclick,/m/0bt9lr,1,0.000000,0.651875,0.000000,0.999062,1,1,0,0,0,0.312500,0.000000,0.317500,0.651875,0.000000,0.410882,0.999062,0.999062
2,0005a9520eb22c19,xclick,/m/0bt9lr,1,0.094167,0.611667,0.055626,0.998736,1,1,0,0,0,0.487500,0.611667,0.243333,0.094167,0.055626,0.226296,0.998736,0.305942
3,0006303f02219b07,xclick,/m/0bt9lr,1,0.000000,0.999219,0.000000,0.998824,1,1,0,0,0,0.508594,0.999219,0.000000,0.478906,0.000000,0.375294,0.720000,0.998824
4,00064d23bf997652,xclick,/m/0bt9lr,1,0.240938,0.906183,0.000000,0.694286,0,0,0,0,0,0.678038,0.906183,0.240938,0.522388,0.000000,0.370000,0.424286,0.694286


In [ ]:
df2['cdlabel'] = df2.apply(lambda row: label_img(row), axis=1)
df2['cdlabel'].unique()

array([1, 0])

In [ ]:
df3 = pd.concat([df2['ImageID'], df2['cdlabel']], axis=1, keys=['ImageID', 'label'])

In [ ]:
df3.head()

,ImageID,label
0,0000b9fcba019d36,1
1,0000cb13febe0138,1
2,0005a9520eb22c19,1
3,0006303f02219b07,1
4,00064d23bf997652,1


In [ ]:
df3.to_csv(r'train_csv.csv', index=False, header=True)

# PyTorch Implementation

In [ ]:
# Configure device for GPU or CPU depending on what is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Old Custom Dataset

In [ ]:
# #PyTorch expects a dataset class to call its methods
# ##some of the finer details can be found here <https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html>
 
# class CustomDataset(Dataset):
#     def __init__(self, csv_file, root_dir, transform):
#         self.root_dir = root_dir
#         self.samples = pd.read_csv(csv_file)
#         #torch pipeline to transform images to tensors
#         self.transform = transform
 
#     def __len__(self):
#         return len(self.samples)
 
#     #Return the image(as a tensor) and the target(dictionary of tensors)
#     def __getitem__(self, idx):
#       cat_label = '/m/01yrx'
#       img_name = os.path.join(self.root_dir, self.samples.iloc[idx,0]+'.jpg')
#       image = Image.open(img_name).convert('RGB')   # OR: image = io.imread(img_name)?
      
#       if self.transform:
#         image = self.transform(image)
   
#       if self.samples.iloc[idx,2] == cat_label:
#         label = torch.as_tensor(0, dtype = torch.int64)
#       else:
#         label = torch.as_tensor(1, dtype = torch.int64)
     
#       image_id = torch.as_tensor(idx, dtype = torch.int64)
      
#       ##save that box as a tensor
#       box = torch.as_tensor([self.samples.iloc[idx, 4]
#              , self.samples.iloc[idx, 6]
#              , self.samples.iloc[idx, 5]
#              , self.samples.iloc[idx, 7]], dtype = torch.float32)
#       area = [(self.samples.iloc[idx, 5] - self.samples.iloc[idx, 4]) * (self.samples.iloc[idx, 7] - self.samples.iloc[idx, 6])]
#       area_at = torch.as_tensor(area, dtype = torch.float32)
     
#       #iscrowd = torch.zeros((1,), dtype=torch.int64)
      
#       target = {}
#       target['boxes'] = box
#       target['labels'] = label
#       target['image_id'] = image_id
#       target['area'] = area_at
#     #   target['iscrowd'] = iscrowd
      
#       return image, target

# Create Custom Dataset

In [ ]:
class CustomDataload(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id+'.jpg')).convert("RGB")
        y_label = torch.tensor(self.annotations.iloc[index, 1])
       
        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)


In [ ]:
#Set training set transform process on PIL Image item from custom dataset 
train_transform = transforms.Compose([
    # transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    # transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

# Set test set transformations:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


train_file = 'train_csv.csv'
img_dir = '/content/drive/MyDrive/Colab Notebooks/CatsNDogs/data/cadod/'

all_data = CustomDataload(root_dir=img_dir, annotation_file=train_file, transform=train_transform)

In [ ]:
batch_size = 64
# shuffle = True
# test_split = .2
 
# dataset_size = len(all_data)
# indices = list(range(dataset_size))
 
# split = int(np.floor(test_split * dataset_size))
# np.random.seed(27)
# np.random.shuffle(indices)
 
# train_indices, test_indices = indices[split:], indices[:split]
 
# #call the subset sampler to sample our data
# train_sampler = SubsetRandomSampler(train_indices) #random sample for the indices for training data and test data
# test_sampler = SubsetRandomSampler(test_indices)

num_train = int(len(all_data) * 0.8)
train_set, test_set = torch.utils.data.random_split(all_data, [num_train, len(all_data) - num_train])

# Make Train Loader:
train_dataloader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
                                  
# Make test loader:
test_dataloader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

# Define the Model

In [ ]:
class Unit(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Unit, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels, kernel_size=3, 
                              out_channels=out_channels, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)

        return output

class ImgNet(nn.Module):
    def __init__(self, num_classes):
        super(ImgNet, self).__init__()

         #Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(in_channels=3,out_channels=32)
        self.unit2 = Unit(in_channels=32, out_channels=32)
        self.unit3 = Unit(in_channels=32, out_channels=32)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)
        
        #Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, 
                                 self.unit4, self.unit5, self.unit6 
                                 ,self.unit7, self.pool2, self.unit8, 
                                 self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)
        
        self.fc = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, input):
        output = self.net(input)
        output = output.view(-1,128)
        output = self.fc(output)
        return output


# Train Model

In [ ]:
from torch.optim import Adam

cuda_avail = torch.cuda.is_available()

model = ImgNet(num_classes=2)

if cuda_avail:
    model.cuda()

optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for name, param in model.named_parameters():
  print(name, '\t', param.shape)

unit1.conv.weight 	 torch.Size([32, 3, 3, 3])
unit1.conv.bias 	 torch.Size([32])
unit1.bn.weight 	 torch.Size([32])
unit1.bn.bias 	 torch.Size([32])
unit2.conv.weight 	 torch.Size([32, 32, 3, 3])
unit2.conv.bias 	 torch.Size([32])
unit2.bn.weight 	 torch.Size([32])
unit2.bn.bias 	 torch.Size([32])
unit3.conv.weight 	 torch.Size([32, 32, 3, 3])
unit3.conv.bias 	 torch.Size([32])
unit3.bn.weight 	 torch.Size([32])
unit3.bn.bias 	 torch.Size([32])
unit4.conv.weight 	 torch.Size([64, 32, 3, 3])
unit4.conv.bias 	 torch.Size([64])
unit4.bn.weight 	 torch.Size([64])
unit4.bn.bias 	 torch.Size([64])
unit5.conv.weight 	 torch.Size([64, 64, 3, 3])
unit5.conv.bias 	 torch.Size([64])
unit5.bn.weight 	 torch.Size([64])
unit5.bn.bias 	 torch.Size([64])
unit6.conv.weight 	 torch.Size([64, 64, 3, 3])
unit6.conv.bias 	 torch.Size([64])
unit6.bn.weight 	 torch.Size([64])
unit6.bn.bias 	 torch.Size([64])
unit7.conv.weight 	 torch.Size([64, 64, 3, 3])
unit7.conv.bias 	 torch.Size([64])
unit7.bn.weight 	 t

In [ ]:
# Check for available GPU for modelNN:
if torch.cuda.is_available():
    model.cuda()

**Maybe a learning rate adjuster??**

In [ ]:
# https://heartbeat.fritz.ai/basics-of-image-classification-with-pytorch-2f8973c51864

# # Create a learning rate adjustment function that divides the learning rate by 10 every 30 epochs
# def adjust_learning_rate(epoch):
#     lr = 0.001

#     if epoch > 180:
#         lr = lr / 1000000
#     elif epoch > 150:
#         lr = lr / 100000
#     elif epoch > 120:
#         lr = lr / 10000
#     elif epoch > 90:
#         lr = lr / 1000
#     elif epoch > 60:
#         lr = lr / 100
#     elif epoch > 30:
#         lr = lr / 10

#     for param_group in optimizer.param_groups:
#         param_group["lr"] = lr


### Test Function

In [ ]:
def save_models(epoch):
    torch.save(model.state_dict(), "cadodModel_{}.model".format(epoch))
    print("Checkpoint saved")

In [ ]:
def test():
    model.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(test_dataloader):

        if cuda_avail:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)

        test_acc += torch.sum(prediction == labels.data)

    test_acc = test_acc / 2593

    return test_acc

# Train Function

In [ ]:
# https://heartbeat.fritz.ai/basics-of-image-classification-with-pytorch-2f8973c51864

def train(num_epochs):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        for i, (images, labels) in enumerate(train_dataloader):
            # Move images and labels to gpu if available
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = loss_fn(outputs, labels)
            # Backpropagate the loss
            loss.backward()

            # Adjust parameters according to the computed gradients
            optimizer.step()

            train_loss += loss.cpu().data * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += torch.sum(prediction == labels.data)

        # Call the learning rate adjustment function
        # adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / num_train
        train_loss = train_loss / num_train

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
            save_models(epoch)
            best_acc = test_acc

        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch,
                                                                                        train_acc,
                                                                                        train_loss,
                                                                                        test_acc))
                        

# Run Tests

In [ ]:
if __name__ == '__main__':
    train(35)



Checkpoint saved
Epoch 0, Train Accuracy: 0.6045121550559998 , TrainLoss: 0.6590253114700317 , Test Accuracy: 0.5715387463569641
Checkpoint saved
Epoch 1, Train Accuracy: 0.6396066546440125 , TrainLoss: 0.6375046968460083 , Test Accuracy: 0.6263015866279602
Checkpoint saved
Epoch 2, Train Accuracy: 0.6699768900871277 , TrainLoss: 0.6082920432090759 , Test Accuracy: 0.6556112766265869
Checkpoint saved
Epoch 3, Train Accuracy: 0.6849209666252136 , TrainLoss: 0.5898261666297913 , Test Accuracy: 0.6826070547103882
Checkpoint saved
Epoch 4, Train Accuracy: 0.7014076709747314 , TrainLoss: 0.5684621334075928 , Test Accuracy: 0.6976475119590759
Epoch 5, Train Accuracy: 0.719244122505188 , TrainLoss: 0.5496002435684204 , Test Accuracy: 0.5696104764938354
Checkpoint saved
Epoch 6, Train Accuracy: 0.7399730086326599 , TrainLoss: 0.5256680250167847 , Test Accuracy: 0.710374116897583
Epoch 7, Train Accuracy: 0.7544350624084473 , TrainLoss: 0.4973798096179962 , Test Accuracy: 0.6980332136154175
Chec

### Olde Train Function

In [ ]:
# # Train function from lab demo
# def train(net, trainloader, testloader, epochs, log_dir1=None, log_dir2=None):
#   writer = SummaryWriter(log_dir1)
#   writer2 = SummaryWriter(log_dir2)

#   criterion1 = nn.NLLLoss()
#   criterion2 = nn.MSELoss()

#   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#   optimizer = Adam(net.parameters(), lr=0.001, weight_decay=0)
#   net.to(device)

# #   # Check for available GPU for modelNN:
# #   if torch.cuda.is_available():
# #       net.to(device)

#   train_accuracy = [0] * epochs
#   test_accuracy = [0] * epochs
#   train_loss = [0] * epochs
#   test_loss = [0] * epochs

#   for epoch in tqdm(range(epochs)):
#     correct = 0
#     total = 0
#     running_loss = 0

#     correct_val = 0
#     total_val=0
#     running_loss_val = 0

#     for batch_idx, (images, data) in enumerate(trainloader):

#         inputs, labels, bbox = images.to(device), data['labels'].to(device), data['boxes'].to(device)
#         optimizer.zero_grad()

#         outputs1, outputs2 = net(inputs) #forward pass # classification # regression

#         _, predicted = torch.max(outputs1.data,1) #class
#         predicted.to(device)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#         CXE = criterion1(outputs1, labels) #NLL
#         MSE = criterion2(outputs2, bbox.float()) #MSE

#         # add weights
#         loss = CXE + MSE # combined loss, add a multiplier to either to weight.
#         loss.backward() #backward pass
#         optimizer.step()
#         running_loss += loss.item()

#     #validation
#     with torch.no_grad():
#       for data_val in testloader:
#         images_val, labels_val, other_val = data_val[0].to(device), data_val[1].to(device), data_val[2].to(device)
#         outputs_val1, outputs_val2 = net(images_val) #forward pass
#         _, predicted_val = torch.max(outputs_val1.data, 1)
#         predicted_val.to(device)
#         total_val += labels_val.size(0)
#         correct_val += (predicted_val == labels_val).sum().item()
#         loss_val1 = criterion1(outputs_val1, labels_val)
#         loss_val2 = criterion2(outputs_val2, other_val)
#         loss_val = loss_val1 + loss_val2
#         running_loss_val += loss_val.item()

#     train_accuracy[epoch] += 100 * correct / total # get train data accuracy
#     test_accuracy[epoch] += 100 * correct_val/total_val # get test data accuracy

#     train_loss[epoch] += running_loss # get train data cross entropy loss
#     test_loss[epoch] += running_loss_val # get test data cross entropy loss

#     writer.add_scalar('accuracy', train_accuracy[epoch], epoch)
#     writer2.add_scalar('accuracy', test_accuracy[epoch], epoch)
  


In [ ]:
train(modelnet, train_dataloader, test_dataloader, 10, log_dir1=None, log_dir2=None)

RuntimeError: ignored

In [ ]:
# def train(epochs):
#     best_acc = 0.0

#     for epoch in range(epochs):
#         modelnet.train()
#         train_acc = 0.0
#         train_loss = 0.0
#         for i, (images, labels) in enumerate(train_dataloader):
#             if torch.cuda.is_available():
#                 images = Variable(images.cuda())
#                 labels = Variable(labels(i)[1]['labels'].cuda())

#             optimize.zero_grad()
#             outputs = modelnet(images)
#             loss = loss_fn(outputs, labels)
#             loss.backward()

#             optimize.step()

#             train_loss += loss.cpu().data[0] + images.size(0)
#             _, prediction = torch.max(outputs.data, 1)

#             train_acc += torch.sum(prediction == labels.data)
        
#         train_acc = train_acc / 12966
#         train_loss = train_loss / 12996

#         test_acc = test()

#         if test_acc > best_acc:
#             save_models(epoch)
#             best_acc = test_acc

#         print("Epoch {}, Train Acc: {} , TrainLoss: {}, Test Acc: {}".format(epoch, train_acc, train_loss))

In [ ]:
if __name__ == "__main__":
    train(train_dataloader, test_dataloader, 20)

TypeError: ignored